In [329]:
import networkx as nx
from node2vec import Node2Vec
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations

In [417]:
n_node = 10000

ring_lattice = nx.cycle_graph(n_node)
ring_missing = nx.cycle_graph(n_node)
ring_missing.remove_edge(0, n_node - 1)

# my_edges = list(ring_lattice.edges)
# my_edges[0][0]

In [465]:
def remove_links(G, n_missing):
    
    my_edges = np.array(list(G.edges))
    number_of_rows = my_edges.shape[0]
    rnd_idx = np.random.choice(number_of_rows,
                               size = n_missing,
                               replace = False)

    pair = my_edges[rnd_idx, :]
    
    for p in pair:
        G.remove_edge(p[0], p[1])
        

In [466]:
n_rm = 20
ring_missing = nx.cycle_graph(n_node)

remove_links(ring_missing, n_rm)

In [348]:
dim = 100

n2v_ring = Node2Vec(ring_lattice, 
                    dimensions = 128, 
                    walk_length = 30,
                    num_walks = 10,
                    workers = 4,
                    )


n2v_ring2 = Node2Vec(ring_lattice, 
                    dimensions = 128, 
                    walk_length = 30,
                    num_walks = 10,
                    workers = 4,
                    )

Generating walks (CPU: 2): 100%|██████████| 3/3 [00:35<00:00, 11.84s/it]


In [349]:
n2v_missing = Node2Vec(ring_missing, 
                       dimensions = 128,
                       walk_length = 30,
                       num_walks = 10,
                       workers = 4,
                       )

Generating walks (CPU: 2): 100%|██████████| 3/3 [00:32<00:00, 10.93s/it]


In [350]:
embed_ring = n2v_ring.fit(window = 10,
                         min_count = 1,
                         batch_words = 4,
                         )

embed_ring2 = n2v_ring2.fit(window = 10,
                         min_count = 1,
                         batch_words = 4,
                         )

embed_missing = n2v_missing.fit(window = 10, 
                                min_count = 1, 
                                batch_words = 4,)

In [351]:
# embed_ring.wv.save_word2vec_format('./ring_embed')

In [352]:
# embed_missing.wv.save_word2vec_format('./missing_embed')

In [353]:
vec_ring = []
vec_ring2 = []
vec_miss = []
for i in range(n_node):
    vec_ring.append(embed_ring.wv['%d'%i])
    vec_ring2.append(embed_ring2.wv['%d'%i])
    vec_miss.append(embed_missing.wv['%d'%i])


In [354]:
# vec_ring = []

# file = open('./ring_embed', 'r')
# for line in file:
#     val = line.split(' ')
#     val[-1] = val[-1].strip('\n')
#     val = list(map(float, val))
#     vec_ring.append(val)


# vec_missing = []

# file = open('./missing_embed', 'r')
# for line in file:
#     val = line.split(' ')
#     val[-1] = val[-1].strip('\n')
#     val = list(map(float, val))
#     vec_missing.append(val)

    
# vec_ring = vec_ring[1:]
# vec_miss = vec_missing[1:]

In [355]:
# a = combinations(range(0, n_node), 2)
# pair = [i for i in a]
# dist_ring = 0.
# dist_miss = 0.
# cosim_ring = 0.
# cosim_miss = 0.

# for p in pair:
#     dist_ring += np.linalg.norm(np.array(vec_miss[p[0]]) - np.array(vec_ring[p[1]]))
#     dist_miss += np.linalg.norm(np.array(vec_miss[p[0]]) - np.array(vec_miss[p[1]]))
    
#     cosim_ring += \
#     np.dot(vec_ring[p[0]], vec_ring[p[1]]) / (np.linalg.norm(vec_ring[p[0]]) * np.linalg.norm(vec_ring[p[1]]))
    
#     cosim_miss += \
#     np.dot(vec_miss[p[0]], vec_miss[p[1]]) / (np.linalg.norm(vec_miss[p[0]]) * np.linalg.norm(vec_miss[p[1]]))
    

# dist_ring = dist_ring / len(pair)
# dist_miss = dist_miss / len(pair)
# cosim_ring = cosim_ring / len(pair)
# cosim_miss = cosim_miss / len(pair)

# print(dist_ring)
# print(dist_miss)

# print(cosim_ring)
# print(cosim_miss)

In [356]:
dist_control = 0.
cosim_control = 0.

for i in range(n_node):
    dist_control += np.linalg.norm(np.array(vec_ring2[i]) - np.array(vec_ring[i]))
    cosim_control += np.dot(vec_ring[i], vec_ring2[i]) / (np.linalg.norm(vec_ring[i]) * np.linalg.norm(vec_ring2[i]))
    
    
dist_control = dist_control / n_node
cosim_control = cosim_control / n_node

In [357]:
dist_exp = 0.
cosim_exp = 0.

for i in range(n_node):
    dist_exp += np.linalg.norm(np.array(vec_miss[i]) - np.array(vec_ring[i]))
    cosim_exp += np.dot(vec_ring[i], vec_miss[i]) / (np.linalg.norm(vec_ring[i]) * np.linalg.norm(vec_miss[i]))
    
    
dist_exp = dist_exp / n_node
cosim_exp = cosim_exp / n_node

In [358]:
print(dist_control)
print(dist_exp)

9.709897774791717
9.736059147024156


In [359]:
print(cosim_control)
print(cosim_exp)

0.24986675921864807
0.24568584918759298


In [221]:
embed_missing.wv.most_similar('0', topn = 20)

[('1999', 0.9990344643592834),
 ('1998', 0.9979798197746277),
 ('1997', 0.9921969771385193),
 ('1996', 0.977904200553894),
 ('1995', 0.9293650388717651),
 ('1994', 0.8656020760536194),
 ('1993', 0.7891461253166199),
 ('1992', 0.7386296391487122),
 ('1991', 0.6905949115753174),
 ('1990', 0.569789707660675),
 ('1989', 0.5093412399291992),
 ('556', 0.4777481257915497),
 ('555', 0.4773986339569092),
 ('466', 0.47380587458610535),
 ('725', 0.472700297832489),
 ('728', 0.47222787141799927),
 ('553', 0.47165459394454956),
 ('557', 0.4699530005455017),
 ('1295', 0.4699190557003021),
 ('720', 0.46833011507987976)]

In [222]:
embed_missing.wv.most_similar('1')

[('2', 0.9985103607177734),
 ('3', 0.9984375834465027),
 ('4', 0.9908352494239807),
 ('5', 0.9722195863723755),
 ('6', 0.93645840883255),
 ('7', 0.8674032688140869),
 ('8', 0.8126690983772278),
 ('9', 0.7122371792793274),
 ('10', 0.6289634704589844),
 ('11', 0.540838360786438)]